In [27]:
import pandas as pd

df = pd.read_excel("files\GVC - Padrões Globais.xlsx")
df

<>:3: SyntaxWarning: invalid escape sequence '\G'
<>:3: SyntaxWarning: invalid escape sequence '\G'
C:\Users\Luiz Fernando\AppData\Local\Temp\ipykernel_14956\3371848847.py:3: SyntaxWarning: invalid escape sequence '\G'
  df = pd.read_excel("files\GVC - Padrões Globais.xlsx")


,Código,Termo
0,3E 02 00 00 00 00 00,Elementos de manejo do terreno
1,3E 02 02 00 00 00 00,Elementos de terrapleno
2,3E 02 02 02 00 00 00,Solo
3,3E 02 02 04 00 00 00,Camada de solo orgânico
4,3E 02 02 06 00 00 00,Platô
...,...,...
1012,3R 22 06 02 00 00 00,treinado (a)
1013,3R 22 06 06 00 00 00,capacitado (a)
1014,3R 22 10 00 00 00 00,Resultados dos trabalhos de segurança e saúde
1015,3R 22 10 02 00 00 00,sinalizado (a)


In [28]:
codeTermDict: dict[str, str] = {}

for idx, value in df["Código"].items():
    parts = value.split()
    
    while len(parts) < 7:
        parts.append("00")
    
    newValue = "-".join(parts)  # '3E-02-00-00-00-00-00'

    term = df.at[idx, "Termo"]
    codeTermDict[newValue] = term

import json
import os

os.makedirs("files", exist_ok=True)
file_path = os.path.join("files", "nested_codes.json")

with open(file_path, "w", encoding="utf-8") as f:
    json.dump(codeTermDict, f, indent=2, ensure_ascii=False)

codeTermDict

{'3E-02-00-00-00-00-00': 'Elementos de manejo do terreno',
 '3E-02-02-00-00-00-00': 'Elementos de terrapleno',
 '3E-02-02-02-00-00-00': 'Solo',
 '3E-02-02-04-00-00-00': 'Camada de solo orgânico',
 '3E-02-02-06-00-00-00': 'Platô',
 '3E-02-02-08-00-00-00': 'Faixa terraplanada',
 '3E-02-04-00-00-00-00': 'Elementos de contenção',
 '3E-02-04-02-00-00-00': 'Muro de arrimo',
 '3E-02-04-02-02-00-00': 'Muro de arrimo por gravidade',
 '3E-02-04-02-04-00-00': 'Muro de arrimo por flexão',
 '3E-02-04-02-06-00-00': 'Muro de gabião',
 '3E-02-04-02-08-00-00': 'Muro de flexão',
 '3E-02-04-04-00-00-00': 'Solo grampeado',
 '3E-02-04-06-00-00-00': 'Cortina de contenção',
 '3E-02-04-06-02-00-00': 'Contenção escorada',
 '3E-02-04-06-04-00-00': 'Contenção não escorada',
 '3E-02-04-06-06-00-00': 'Berma',
 '3E-02-04-06-08-00-00': 'Cortina de contenção com pranchão',
 '3E-02-04-06-10-00-00': 'Cortina de contenção com estacas justapostas',
 '3E-02-04-06-12-00-00': 'Cortina de contenção com parede diafragma',
 '3

In [34]:
from typing import List, Dict
from fuzzywuzzy import fuzz, process
from Types import GvcObject, Property
import re
from sentence_transformers import SentenceTransformer, util

EMBED_MODEL = SentenceTransformer('all-MiniLM-L6-v2')

IGNORED_KEYWORDS = [
    "id", "guid", "handle", "path", "element", "category id",
    "family id", "internal", "level", "host", "parameter id", "unique id", "layer",
    "required", "icon"
]

NUMERIC_PATTERN = re.compile(r"^\d+([.,]\d+)?$")

def is_useful_property(prop: Property) -> bool:
    if not prop or not prop.Info:
        return False
    if len(prop.Info.strip()) < 3:
        return False
    if NUMERIC_PATTERN.match(prop.Info.strip()):
        return False
    name = (prop.Name or "").lower()
    cat = (prop.Category or "").lower()
    if any(kw in name for kw in IGNORED_KEYWORDS):
        return False
    if any(kw in cat for kw in IGNORED_KEYWORDS):
        return False
    return True

def detect_code(obj: GvcObject,codes_dict: Dict[str, str], fuzzy_weight: float = 0.7, semantic_weight: float = 0.3, min_score: float = 50, use_fuzzy: bool = True, use_semantic: bool = True) -> Dict[str, any]:

    props_text = [p.Info for p in obj.Properties if is_useful_property(p)]

    if not props_text:
        return {
            "object_id": obj.ObjectId,
            "name": obj.Name,
            "match_code": None,
            "description": None,
            "score": 0
        }

    combined_text = " ".join(props_text).lower().strip()
    if not combined_text:
        return {
            "object_id": obj.ObjectId,
            "name": obj.Name,
            "match_code": None,
            "description": None,
            "score": 0
        }

    descriptions = list(codes_dict.values())

    if use_fuzzy:
        fuzzy_scores_list = [fuzz.token_set_ratio(combined_text, d) for d in descriptions]
    else:
        fuzzy_scores_list = [0] * len(descriptions)

    if use_semantic:
        obj_embedding = EMBED_MODEL.encode(combined_text, convert_to_tensor=True)
        desc_embeddings = EMBED_MODEL.encode(descriptions, convert_to_tensor=True)
        semantic_scores_list = (util.pytorch_cos_sim(obj_embedding, desc_embeddings)[0] * 100).tolist()
    else:
        semantic_scores_list = [0] * len(descriptions)

    final_scores = {}
    for desc, f_score, s_score in zip(descriptions, fuzzy_scores_list, semantic_scores_list):
        final_scores[desc] = f_score * fuzzy_weight + s_score * semantic_weight

    best_final_desc = max(final_scores, key=final_scores.get)
    final_score = final_scores[best_final_desc]

    if final_score < min_score:
        return {
            "object_id": obj.ObjectId,
            "name": obj.Name,
            "match_code": None,
            "description": None,
            "score": final_score
        }

    match_code = next((k for k, v in codes_dict.items() if v == best_final_desc), None)

    return {
        "object_id": obj.ObjectId,
        "name": obj.Name,
        "match_code": match_code,
        "description": best_final_desc,
        "score": final_score
    }

def detect_all_codes(items: List[GvcObject], codes_dict: Dict[str, str]):
    results = []
    for obj in items:
        match = detect_code(obj, codes_dict , 0.4, 0.6, 40)
        results.append(match)
    return results


In [35]:
from Db import DBManager
import json

items = DBManager._getItems(limit=20)
matches = detect_all_codes(items, codeTermDict)

for m in matches:
    print(f"[{m['score']}%] {m['name']} → {m['match_code']} | {m['description']}")


Status: 200


Processando itens: 100%|██████████| 20/20 [00:00<00:00, 630.03it/s]


[53.670149612426755%] Parede básica → 3E-14-02-00-00-00-00 | Parede
[54.257344818115236%] Parede básica → 3E-14-02-00-00-00-00 | Parede
[54.01464195251465%] Parede básica → 3E-14-02-00-00-00-00 | Parede
[62.05213088989258%] MTZ_RE_EQ_LAVATORIO COM COLUNA SUSPENSA → 3E-38-06-06-00-00-00 | Coluna de água
[54.530540466308594%] Parede básica → 3E-14-02-00-00-00-00 | Parede
[57.36756057739258%] Parede básica → 3E-14-02-00-00-00-00 | Parede
[48.768035888671875%] ARQ_Metais_Sifao_Lavatorio → 3E-36-14-00-00-00-00 | Elementos para distribuição de água potável
[55.314840698242186%] Parede básica → 3E-14-02-00-00-00-00 | Parede
[61.90248794555664%] Piso → 3E-16-02-00-00-00-00 | Pisos
[53.545538330078124%] Parede básica → 3E-14-02-00-00-00-00 | Parede
[62.018883514404294%] ARQ_Porta_Madeira_1F_Abrir → 3E-20-02-02-00-00-00 | Portas
[47.10422554016114%] Celite_Tanque M_51203 → 3E-38-06-06-00-00-00 | Coluna de água
[52.55984611511231%] ARQ_Janela_Alumínio_1F_Fixa_Ventilação → 3E-20-04-02-00-00-00 | J